In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os
import sys

import helper

from selenium.common.exceptions import TimeoutException

In [2]:
course_schedule = []
NO_WINDOW = False
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

In [3]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

In [4]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)

In [5]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)


In [6]:
def sign_in(driver, wait):
    
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092361")
    password.send_keys("LP7J")
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [7]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [8]:
with open(r'./class_schedules/15 February - 21 February 2021.json', "r") as read_file:
    course_schedule = json.load(read_file)

In [9]:
# course_schedule

In [10]:
# NUM_CLASSES = 1
# DELAY_BETWEEN_CLASSSES = 3
# BASE_DELAY = 5
# TEST_SIGNIN_SECONDS_DELAY_FROM_NOW = BASE_DELAY
# TEST_BOOK_CLASSES_SECONDS_DELAY_FROM_NOW =  (NUM_CLASSES-1) * DELAY_BETWEEN_CLASSSES + BASE_DELAY*2

# signin_datetime = datetime.datetime.now() + datetime.timedelta(seconds=TEST_SIGNIN_SECONDS_DELAY_FROM_NOW)
# t = datetime.datetime.now()
# future_now = datetime.datetime(t.year, t.month, t.day, t.hour, t.minute + 1, 0)
# book_datetime = future_now + datetime.timedelta(seconds=TEST_BOOK_CLASSES_SECONDS_DELAY_FROM_NOW)

# print("Now: {}".format(datetime.datetime.now()))
# print("book_datetime: {}".format(book_datetime))
# courses_to_be_booked = []

# c = helper.search_courses_by_date('27', course_schedule)
# c = helper.search_courses_by_name('Mysore', c)
# # c = helper.search_courses_by_time('AM', c)


# courses_to_be_booked.extend(c)

# helper.list_courses_to_be_booked(courses_to_be_booked)

In [11]:
YEAR = 2021
MONTH = 2
Day = 12

book_datetime = datetime.datetime(YEAR, MONTH, Day, 22, 0, 0)

temp = []

c = helper.search_courses_by_date('16', course_schedule)
c = helper.search_courses_by_name('Counter', c)
c = helper.search_courses_by_teacher('Marcus', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

# c = helper.search_courses_by_date('22', course_schedule)
# c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Simon', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# # Monday
# #
# c = helper.search_courses_by_date('8', course_schedule)
# # c = helper.search_courses_by_name('Core', c)
# c = helper.search_courses_by_teacher('Ethan', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('8', course_schedule)
# # c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('8', course_schedule)
# c = helper.search_courses_by_name('Hot', c)
# c = helper.search_courses_by_teacher('Tumai', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# # Tuesday
# #

# c = helper.search_courses_by_date('9', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Marcus', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# #

# # Wednesday 
# #

# c = helper.search_courses_by_date('10', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Jun', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #


# # Thursday
# #

# c = helper.search_courses_by_date('4', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('4', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Ethan', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# # Friday
# #

# c = helper.search_courses_by_date('5', course_schedule)
# c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# # Saturday
# #

# c = helper.search_courses_by_date('6', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Simon', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('6', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Anita', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #


# # Sunday
# #
# c = helper.search_courses_by_date('7', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('YaYa', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)


# c = helper.search_courses_by_date('7', course_schedule)
# c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('7', course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #


# c = helper.search_courses_by_date('14', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Andy', c)
# # c = helper.search_courses_by_time('', c)
# temp.extend(c)



# c = helper.search_courses_by_date('13', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Jun', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('13', course_schedule)
# c = helper.search_courses_by_name('Hot Flow', c)
# c = helper.search_courses_by_teacher('Carolyn', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)



# c = helper.search_courses_by_date('12', course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('11', course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('11', course_schedule)
# c = helper.search_courses_by_name('Hot', c)
# c = helper.search_courses_by_teacher('Tumai', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('11', course_schedule)
# c = helper.search_courses_by_name('My', c)
# c = helper.search_courses_by_teacher('吳', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('10', course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('08', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Andy', c)
# c = helper.search_courses_by_time('04:00', c)
# temp.extend(c)

# c = helper.search_courses_by_date('08', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Andy', c)
# c = helper.search_courses_by_time('05:15', c)
# temp.extend(c)

# c = helper.search_courses_by_date('08', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Andy', c)
# c = helper.search_courses_by_time('06:45', c)
# temp.extend(c)

# c = helper.search_courses_by_date('06', course_schedule)
# c = helper.search_courses_by_name('Aerial', c)
# c = helper.search_courses_by_teacher('Joe', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('02', course_schedule)
# c = helper.search_courses_by_name('Aerial', c)
# c = helper.search_courses_by_teacher('Sankari', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('02', course_schedule)
# # c = helper.search_courses_by_name('Aerial', c)
# c = helper.search_courses_by_teacher('Simon', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('01', course_schedule)
# c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Andy', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)


# c = helper.search_courses_by_date('29', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('29', course_schedule)
# c = helper.search_courses_by_name('Intro', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('26', course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Vicky', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)



# c = helper.search_courses_by_date('25', course_schedule)
# c = helper.search_courses_by_name('Yoga', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('25', course_schedule)
# c = helper.search_courses_by_name('Yin', c)
# c = helper.search_courses_by_teacher('Priya', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('23', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Jun', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('18', course_schedule)
# c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('18', course_schedule)
# c = helper.search_courses_by_name('Wheel', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)


# c = helper.search_courses_by_date('18', course_schedule)
# c = helper.search_courses_by_name('Tone', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('17', course_schedule)
# c = helper.search_courses_by_name('Aerial', c)
# c = helper.search_courses_by_teacher('Vicky', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# # c = helper.search_courses_by_date('14', course_schedule)
# # c = helper.search_courses_by_name('Myofascial', c)
# # c = helper.search_courses_by_teacher('吳', c)
# # c = helper.search_courses_by_time('PM', c)
# # temp.extend(c)

# c = helper.search_courses_by_date('14', course_schedule)
# # c = helper.search_courses_by_name('Hot', c)
# c = helper.search_courses_by_teacher('Tumai', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('15', course_schedule)
# c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('11', course_schedule)
# c = helper.search_courses_by_name('Yin Yoga', c)
# # c = helper.search_courses_by_teacher('', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('23', course_schedule)
# c = helper.search_courses_by_name('Yin', c)
# c = helper.search_courses_by_teacher('Priya', c)
# c = helper.search_courses_by_time('PM', c)

# temp.extend(c)

# c = helper.search_courses_by_date('03', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Vicky', c)
# c = helper.search_courses_by_time('PM', c)

# temp.extend(c)

# c = helper.search_courses_by_date('18', course_schedule)
# c = helper.search_courses_by_name('Basic', c)
# c = helper.search_courses_by_teacher('Yvette', c)
# # c = helper.search_courses_by_time('PM', c)

# temp.extend(c)

# c = helper.search_courses_by_date('08', course_schedule)
# # c = helper.search_courses_by_name('Hot', c)
# c = helper.search_courses_by_teacher('Tumai', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('04', course_schedule)
# c = helper.search_courses_by_name('壁', c)
# # c = helper.search_courses_by_teacher('Yvette', c)
# # c = helper.search_courses_by_time('PM', c)

# temp.extend(c)

# c = helper.search_courses_by_date('11', course_schedule)
# c = helper.search_courses_by_name('Basic', c)
# c = helper.search_courses_by_teacher('Yvette', c)
# c = helper.search_courses_by_time('PM', c)

# temp.extend(c)

# c = helper.search_courses_by_date('24', course_schedule)
# # c = helper.search_courses_by_name('Basic', c)
# c = helper.search_courses_by_teacher('Marcus', c)
# # c = helper.search_courses_by_time('PM', c)

# temp.extend(c)



courses_to_be_booked = temp

# courses_to_be_booked = []
# courses_to_be_booked.append(temp[3])
# courses_to_be_booked.append(temp[0])
# courses_to_be_booked.append(temp[1])   
# courses_to_be_booked.append(temp[2])

# courses_to_be_booked = []

# course = {}
# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=747149'
# course['name'] = '艾楊格 壁繩'
# course['teacher'] = 'Yvette'
# course['date'] = '19 Oct Mon'
# course['time'] = '07:15 - 08:45 PM'
# courses_to_be_booked.append(course)

# course = {}
# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=747148'
# course['name'] = 'Iyengar Basic 艾揚格基礎'
# course['teacher'] = 'Yvette'
# course['date'] = '19 Oct Mon'
# course['time'] = '06:00 - 07:00 PM'
# courses_to_be_booked.append(course)

# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=744153'
# course['name'] = 'Universal 1'
# course['teacher'] = 'YaYa'
# course['date'] = '18 Oct Sun'
# course['time'] = '02:00 - 03:30 PM'
# courses_to_be_booked.append(course)

# course = {}
# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=744154'
# course['name'] = 'Yin Yoga 陰瑜珈'
# course['teacher'] = 'YaYa'
# course['date'] = '18 Oct Sun'
# course['time'] = '03:45 - 04:45 PM'
# courses_to_be_booked.append(course)

helper.list_courses_to_be_booked(courses_to_be_booked)

# Set Sign In time 10 minutes before the open for booking time
signin_datetime = book_datetime - datetime.timedelta(seconds=600)

Date: 16 Feb Tue
-----------------------

Name: Counter Flow 1
Teacher: Marcus
Time: 03:30 - 04:30 PM




In [12]:
book_datetime = book_datetime + datetime.timedelta(seconds=0.45)

In [13]:
def scheduled_sign_in(driver, wait):
#     print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [14]:
reserve_class_log_dicts = []

def reserve_class(driver, wait, course_to_be_booked):
    
    print("reserve_class: ({})\n".format(course_to_be_booked['time']))
    
    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    
    reserve_class_log_dict = {}
    
    reserve_class_log_dict['name'] = class_name
    
    now1 = datetime.datetime.now()
    driver.get(book_url)
    now2 = datetime.datetime.now()

    now3 = datetime.datetime.now()
    now4 = datetime.datetime.now()
    if click_book_this_class_now(driver, wait):
        now4 = datetime.datetime.now()
    else:
        now4 = datetime.datetime.now()

    reserve_class_log_dict['datetime1'] = now1
    reserve_class_log_dict['datetime2'] = now2
    reserve_class_log_dict['datetime3'] = now3
    reserve_class_log_dict['datetime4'] = now4
    
    reserve_class_log_dicts.append(reserve_class_log_dict)
    

In [15]:
def show_all_booking_result(reserve_class_log_dicts):
    
    print("Scheduling Logs:")
    
    for reserve_class_log_dict in reserve_class_log_dicts:

        print("Class: ", reserve_class_log_dict['name'])
        print("-----------------------------------------\n")
        print("\t** Begin: Go to the class page at: {}\n".format(reserve_class_log_dict['datetime1']))
        print("\t** End: Go to the class page at: {}\n".format(reserve_class_log_dict['datetime2']))

        print("\t**Begin: Click 'BOOK THIS CLASS NOW' button at: {}\n".format(reserve_class_log_dict['datetime3']))
        print("\t**End: Click 'BOOK THIS CLASS NOW' button at: {}\n".format(reserve_class_log_dict['datetime4']))
        print("-----------------------------------------\n")

In [16]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    signin_datetime_var = signin_datetime + datetime.timedelta(seconds=(idx+1)*1)
    print("idx: ", idx, "scheduled signin datetime: ", signin_datetime_var)
    print("idx: ", idx, "scheduled book datetime: ", book_datetime)
    
    scheduler.add_job(scheduled_sign_in, 'date', run_date=signin_datetime_var, args=[driver, wait])
    scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[driver, wait, course_to_be_booked])
    
    if idx == len(courses_to_be_booked)-1:
        scheduler.add_job(show_all_booking_result, 'date', run_date= book_datetime + datetime.timedelta(seconds=6), args=[reserve_class_log_dicts])

# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()
 


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


 
idx:  0 scheduled signin datetime:  2021-02-12 21:50:01
idx:  0 scheduled book datetime:  2021-02-12 22:00:00.450000


In [17]:
###### print("https://www.trueclassbooking.com.tw/member/login.aspx\n")
for idx, course_to_be_booked in enumerate(courses_to_be_booked):
    print("Class: {} \n\turl: {}\n".format(course_to_be_booked['name'], course_to_be_booked['href']))

Class: Counter Flow 1 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=790298



Run time of job "scheduled_sign_in (trigger: date[2021-02-12 21:50:01 CST], next run at: 2021-02-12 21:50:01 CST)" was missed by 0:00:02.762256


reserve_class: (03:30 - 04:30 PM)

Scheduling Logs:
